In [5]:
import numpy as np
import datetime as dt
import requests
import json
import pandas as pd

In [6]:
url = "https://api.football-data.org/v2/competitions/BSA/matches"
headers = {
    'X-Auth-Token': "220028f7abff4e308cf1e8711ae34a58",
    }
proxies = {
 "http": "http://lanbclb.t-mobile.at:8080",
 "https": "http://lanbclb.t-mobile.at:8080",
}

r = requests.get(url, headers=headers, proxies=proxies)
#r = requests.get(url, headers=headers)
r_json = r.json()

In [7]:
# extract matches from json object
matches_raw = r_json['matches']
df_matches_raw = pd.DataFrame.from_dict(matches_raw)

In [8]:
df_matches_raw

,id,season,utcDate,status,matchday,stage,group,lastUpdated,odds,score,homeTeam,awayTeam,referees
0,297885,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2020-08-08T22:00:00Z,FINISHED,1,REGULAR_SEASON,Regular Season,2020-08-11T07:32:35Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 3984, 'name': 'Fortaleza EC'}","{'id': 1768, 'name': 'CA Paranaense'}","[{'id': 11173, 'name': 'Braulio da Silva Macha..."
1,297879,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2020-08-08T22:30:00Z,FINISHED,1,REGULAR_SEASON,Regular Season,2020-08-11T07:32:35Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 4241, 'name': 'Coritiba FBC'}","{'id': 6684, 'name': 'SC Internacional'}","[{'id': 11426, 'name': 'Bruno Arleu de Araujo'..."
2,297882,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2020-08-09T00:00:00Z,FINISHED,1,REGULAR_SEASON,Regular Season,2020-08-11T07:32:35Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'HOME_TEAM', 'duration': 'REGULAR',...","{'id': 1778, 'name': 'SC Recife'}","{'id': 1837, 'name': 'Ceará SC'}","[{'id': 60939, 'name': 'Edina Alves Batista', ..."
3,297877,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2020-08-09T19:00:00Z,FINISHED,1,REGULAR_SEASON,Regular Season,2020-08-12T09:32:37Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'DRAW', 'duration': 'REGULAR', 'ful...","{'id': 6685, 'name': 'Santos FC'}","{'id': 4286, 'name': 'RB Bragantino'}","[{'id': 11185, 'name': 'Ricardo Marques Ribeir..."
4,297880,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2020-08-09T19:00:00Z,FINISHED,1,REGULAR_SEASON,Regular Season,2020-08-12T09:32:37Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': 'AWAY_TEAM', 'duration': 'REGULAR',...","{'id': 1783, 'name': 'CR Flamengo'}","{'id': 1766, 'name': 'CA Mineiro'}","[{'id': 11260, 'name': 'Raphael Claus', 'natio..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,298251,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2021-02-25T00:00:00Z,SCHEDULED,38,REGULAR_SEASON,Regular Season,2020-07-26T18:40:22Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': None, 'duration': 'REGULAR', 'fullT...","{'id': 1780, 'name': 'CR Vasco da Gama'}","{'id': 4250, 'name': 'Goiás EC'}",[]
376,298252,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2021-02-25T00:00:00Z,SCHEDULED,38,REGULAR_SEASON,Regular Season,2020-07-26T18:40:22Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': None, 'duration': 'REGULAR', 'fullT...","{'id': 1765, 'name': 'Fluminense FC'}","{'id': 3984, 'name': 'Fortaleza EC'}",[]
377,298253,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2021-02-25T00:00:00Z,SCHEDULED,38,REGULAR_SEASON,Regular Season,2020-07-26T18:40:22Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': None, 'duration': 'REGULAR', 'fullT...","{'id': 1768, 'name': 'CA Paranaense'}","{'id': 1778, 'name': 'SC Recife'}",[]
378,298254,"{'id': 589, 'startDate': '2020-08-09', 'endDat...",2021-02-25T00:00:00Z,SCHEDULED,38,REGULAR_SEASON,Regular Season,2020-07-26T18:40:22Z,{'msg': 'Activate Odds-Package in User-Panel t...,"{'winner': None, 'duration': 'REGULAR', 'fullT...","{'id': 1837, 'name': 'Ceará SC'}","{'id': 1770, 'name': 'Botafogo FR'}",[]


In [9]:
# functions to clean data and prepare table MATCHES
def clean_team(row):
    return row['name']

def clean_score_home(row):
    score = row['fullTime']['homeTeam']
    if score is None:
        return -99999
    else:
        return int(score)

def clean_score_away(row):
    score = row['fullTime']['awayTeam']
    if score is None:
        return -99999
    else:
        return int(score)
    
def clean_date(row):
    date_time_str = row
    date_time_obj = dt.datetime.strptime(date_time_str, '%Y-%m-%dT%H:%M:%SZ')
    date_str = date_time_obj.strftime('%d/%m/%Y')
    return date_str

In [10]:
df_matches = pd.DataFrame()

In [11]:
df_matches['MATCH_ID'] = df_matches_raw['id']
df_matches['MATCH_DAY'] = df_matches_raw['matchday']
df_matches['DATE'] = df_matches_raw['utcDate'].apply(clean_date)
df_matches['HOME'] = df_matches_raw['homeTeam'].apply(clean_team)
df_matches['AWAY'] = df_matches_raw['awayTeam'].apply(clean_team)
df_matches['GOALS_HOME'] = df_matches_raw['score'].apply(clean_score_home)
df_matches['GOALS_AWAY'] = df_matches_raw['score'].apply(clean_score_away)

In [12]:
df_matches

,MATCH_ID,MATCH_DAY,DATE,HOME,AWAY,GOALS_HOME,GOALS_AWAY
0,297885,1,08/08/2020,Fortaleza EC,CA Paranaense,0,2
1,297879,1,08/08/2020,Coritiba FBC,SC Internacional,0,1
2,297882,1,09/08/2020,SC Recife,Ceará SC,3,2
3,297877,1,09/08/2020,Santos FC,RB Bragantino,1,1
4,297880,1,09/08/2020,CR Flamengo,CA Mineiro,0,1
...,...,...,...,...,...,...,...
375,298251,38,25/02/2021,CR Vasco da Gama,Goiás EC,-99999,-99999
376,298252,38,25/02/2021,Fluminense FC,Fortaleza EC,-99999,-99999
377,298253,38,25/02/2021,CA Paranaense,SC Recife,-99999,-99999
378,298254,38,25/02/2021,Ceará SC,Botafogo FR,-99999,-99999
